In [1]:
from policyengine_us import Microsimulation
from policyengine_core.reforms import Reform
baseline = Microsimulation(dataset="hf://policyengine/policyengine-us-data/enhanced_cps_2024.h5")


/Users/daphnehansell/miniconda3/envs/policyengine/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
baseline_aca_enrollment = baseline.calculate("has_marketplace_health_coverage", map_to="person", period=2025).sum()
print(baseline_aca_enrollment)

baseline_has_coverage = baseline.calculate("has_marketplace_health_coverage", map_to="person", period=2025)
baseline_is_eligible = baseline.calculate("is_aca_ptc_eligible", map_to="person", period=2025)

baseline_aca = ((baseline_has_coverage & baseline_is_eligible)*baseline_has_coverage.weights).sum()



20285684.746812854


In [9]:
baseline_has_coverage = baseline.calculate("has_marketplace_health_coverage", map_to="person", period=2025)
baseline_is_eligible = baseline.calculate("is_aca_ptc_eligible", map_to="person", period=2025)

baseline_aca = ((baseline_has_coverage & baseline_is_eligible)*baseline_has_coverage.weights).sum()

baseline_aca/1e6

11734775.520414181


TypeError: unsupported operand type(s) for /: 'NoneType' and 'float'